In [137]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
import pandas as pd

In [41]:
def get_website_driver(url):
    driver = webdriver.Chrome()
    driver.get(url)

    return driver

### 溯源農糧產品追溯系統-生產者資訊

In [40]:
def get_search_result(driver, keyword):

    element = driver.find_element(By.NAME, "ByName")
    element.send_keys(keyword)

    search_button = driver.find_element(By.ID, "Search")
    search_button.click()
    rows = driver.find_elements(By.XPATH, "//tbody/tr")

    data = []
    for row in rows[3:8]: 
        columns = row.find_elements(By.TAG_NAME, "td")
        row_data = [col.text.strip() for col in columns] 
        data.append(row_data)
        
    return data

In [42]:
driver = get_website_driver(url="https://qrc.afa.gov.tw/ProducerSearch")

In [43]:
result = get_search_result(driver, keyword='張志銘')

In [44]:
result

[['如欲顯示，請於查詢時輸入相應追溯編號', '張志銘', '荔枝', '臺中市西區', '0'],
 ['如欲顯示，請於查詢時輸入相應追溯編號', '張志銘', '金柑', '宜蘭縣頭城鎮', '19'],
 ['如欲顯示，請於查詢時輸入相應追溯編號', '張志銘-豆漿農場', '稉稻(白米或黑米)', '彰化縣芬園鄉', '23'],
 ['如欲顯示，請於查詢時輸入相應追溯編號 停用', '張志銘', '落花生', '雲林縣崙背鄉', '40'],
 ['如欲顯示，請於查詢時輸入相應追溯編號', '張志銘', '竹筍、鳳梨、甘藷(地瓜)', '新竹縣竹東鎮', '127']]

In [46]:
coa_data = pd.read_csv('./data/crawler_test_data/COA_OpenData.csv')

In [51]:
coa_data[coa_data['producer'] == '張○銘']

,traceCode,producer,address,mark,url,description,status,modifyDate
223,1201004566,張○銘,雲林縣大埤鄉,產銷履歷農產品,https://www.facebook.com/theblessingrice/,這是故事起源也是開始....一個從小沒有接觸過農業家庭的我，卻因為祖先的離去遺留下珍貴的農地...,通過,2025/01/14
1477,401007129,張○銘,臺中市東勢區,NaN,NaN,於東勢區種植苦茶，生產苦茶油，歡迎選購,通過,2024/10/15
4524,701001614,張○銘,宜蘭縣礁溪鄉,NaN,NaN,本人主要種植絲瓜為主，主要銷售為批發市場及自銷方式行銷慢慢建立自有品牌，還有生產絲瓜水及絲瓜...,通過,2024/07/29
9551,401004175,張○銘,臺中市北區,NaN,NaN,台中東勢區龍安橋附近，以自家也能安心食用為初衷。自家栽種未曾使用農藥及化肥。。山上有苦茶、枇...,通過,2024/05/17
9591,701002292,張○銘,宜蘭縣頭城鎮,NaN,NaN,生產高品質農作物,通過,2024/05/16
9878,1201008998,張○銘,雲林縣莿桐鄉,NaN,NaN,種植番茄.,通過,2024/05/08


### Conclusion

Not fit

### 臺灣水產品生產追溯資訊

In [172]:
def get_search_result(driver, keyword):

    element = driver.find_element(By.ID, "QuickSearchByKeyword")
    element.clear()
    element.send_keys(keyword)

    search_button = driver.find_element(By.ID, "quick-search")
    search_button.click()

    time.sleep(5)

    # Find all result rows
    rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'list-body')]")


    tracking_number = None
    producer_name = None
    for row in rows:
        # Get the tracking number from the <div> with data-title="追溯編號"
        tracking_number_div = row.find_element(By.XPATH, ".//div[@data-title='追溯編號']")
        tracking_number = tracking_number_div.text.strip()

        # Get the producer name from the <a> inside <div> with data-title="生產者/單位名稱"
        producer_name_a = row.find_element(By.XPATH, ".//div[@data-title='生產者/單位名稱']/a")
        producer_name = producer_name_a.text.strip()

    return tracking_number, producer_name

In [173]:
driver = get_website_driver(url="https://www.fishqrc.org.tw/")
tracking_number, producer_name = get_search_result(driver, '蔡家順')
driver.quit()

In [167]:
df_w_prod = pd.read_json('./data/crawler_test_data/w_producer.json.txt')
df_w_prod['TraceNo'] = df_w_prod['TraceNo'].astype(str)

In [175]:
df_w_prod[(df_w_prod['TraceNo'] == tracking_number) & (df_w_prod['AquaName'] == producer_name)]

,TraceNo,AquaName,Addr,EMail,Url,ProdName
7383,1311605501,蔡家順,嘉義縣布袋鎮,,,虱目魚、白蝦
